# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


### Libraries

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/CV/Project/Code')

import random
import json
import torch
import pickle
import os
import torch.nn as nn
import numpy as np
from models import GbcNet
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import copy
import time
import math
import cv2
from PIL import Image
import torch.nn.functional as F
from torchvision.transforms import Resize, ToTensor, Compose, GaussianBlur
from torchvision.models.detection import retinanet_resnet50_fpn_v2
from torchvision.models.detection import RetinaNet
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch.nn.init as init
from torch.optim import SGD, Adam, NAdam
#from torchmetrics.detection import MeanAveragePrecision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import one_hot

from sklearn.metrics import roc_auc_score

from torchvision.utils import draw_bounding_boxes
import torchvision
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### File Paths

In [ ]:
!unzip "/content/drive/MyDrive/CV/Project/data/images.zip" -d "/content"
imgs_folder_path = "/content/images"
gold_file_path = "/content/drive/MyDrive/CV/Project/data/gold_boxes.json"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"

### Helper Functions

In [ ]:
def read_txt(txt_path):
  ids = []
  cls = []
  with open(txt_path,"r") as f:
    for line in f.readlines():
      ids.append(line.strip().split(',')[0])
      cls.append(int(line.strip().split(',')[1]))

  return ids, cls

def read_json(json_path):
  with open(json_path, "r") as f:
    json_file = json.load(f)

  return json_file

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize(img, pred_bboxes=None, title=None):
    print("visualize")
    edgecolors = ["y", "r", "c"]
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot the original image with bounding box
    if torch.is_tensor(img):
      ax.imshow(img.permute(1, 2, 0).numpy())
    else:
      ax.imshow(img)
    if pred_bboxes is not None:
      for i in range(len(pred_bboxes)):
        pred_bbox = pred_bboxes[i]
        rect = patches.Rectangle((pred_bbox[0], pred_bbox[1]), pred_bbox[2]-pred_bbox[0], pred_bbox[3]-pred_bbox[1], linewidth=2, edgecolor=edgecolors[i], facecolor='none')
        ax.add_patch(rect)
    ax.set_title(title)

    plt.show()


def gold_box_proportion(x):
  if x < 30:
    return 1
  elif 30 <= x < 35:
    return 0.75
  elif 35 <= x < 40:
    return 0.5
  elif 40 <= x < 45:
    return 0.25
  else:
    return 0

def print_status(txt, status_flag):
  if status_flag==False:
    print(txt)
  return True

def filter_background(predictions):
  # Filter out the predictions with label 0
  filtered_predictions = []
  for prediction in predictions:
      mask = prediction['labels'] != 0
      filtered_prediction = {k: v[mask] for k, v in prediction.items()}
      filtered_predictions.append(filtered_prediction)
  return filtered_predictions

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

### Transforms

In [ ]:
class img_bbox_transform(object):
    def __init__(self, target_width, target_height):
      self.target_width = target_width
      self.target_height = target_height
      self.transforms = Compose([Resize((self.target_width, self.target_height)),
                                ToTensor()])

    def __call__(self, image=None, bbox = None, current_width=None, current_height=None):
      transformed_image = None
      if image is not None:
        current_width, current_height = image.size
        transformed_image = self.transforms(image)
      transformed_bbox = None
      if bbox is not None:
        transformed_bbox = self.transform_bbox(bbox, current_width, current_height)
      return transformed_image, transformed_bbox

    def transform_bbox(self, bbox, current_width=None, current_height=None):
      new_x1 = bbox[0]*self.target_width/current_width
      new_y1 = bbox[1]*self.target_height/current_height
      new_x2 = bbox[2]*self.target_width/current_width
      new_y2 = bbox[3]*self.target_height/current_height
      return [round(new_x1.item()), round(new_y1.item()), round(new_x2.item()), round(new_y2.item())]


def crop_image(image, box, p):
    x1, y1, x2, y2 = box
    # Recieves images in format CHW
    cropped_image = image[:, int((1-p)*y1):int((1+p)*y2), int((1-p)*x1):int((1+p)*x2)]
    return cropped_image

def get_train_transforms(width, height, p=None):
    return A.Compose([
        A.Rotate(limit=5, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.6, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit= 0.1, p=0.5),
        A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=0, p=0.5),
        A.MotionBlur(blur_limit=(3, 5), p=0.5),
        A.Perspective(scale=(0.01, 0.05), keep_size=True, pad_mode=0, pad_val=0, mask_pad_val=0, fit_output=False, interpolation=1, always_apply=False, p=0.7),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
        A.Resize(height=height, width=width, p=1),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

def get_eval_transforms(width, height):
    return A.Compose([
        A.Resize(height=height, width=width, p=1),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

### Dataset

In [ ]:
class InputDataset(Dataset):
    def __init__(self, img_folder, id_txt, gold_json, p=0.15, width=224, height=224, img_transforms=None, do_crop=None, predicted_bboxes_json=None, to_blur=False, blur_kernel_size=(65,65), sigma=0):
        self.img_folder = img_folder
        self.id_txt = id_txt
        self.gold_json = gold_json
        self.to_blur = to_blur
        self.blur_kernel_size = blur_kernel_size
        self.sigma = sigma
        self.do_crop=do_crop
        self.p=p
        self.width=width
        self.height=height
        self.transforms = img_transforms
        self.ids, self.true_labels = read_txt(id_txt)
        self.gold_dict = read_json(gold_json)
        self.predicted_boxes = None
        if predicted_bboxes_json is not None:
          self.predicted_boxes=read_json(predicted_bboxes_json)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_label = self.true_labels[idx]
        gold_bbox = self.gold_dict[img_id]['Gold']
        img_path = f"{self.img_folder}/{img_id}"
        img = cv2.imread(img_path)
        img_height, img_width, _ = img.shape
        if self.to_blur:
          img = cv2.GaussianBlur(img, self.blur_kernel_size, self.sigma)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if (self.do_crop==None) or (self.do_crop=="gold_boxes"):
          transformed_dict = self.transforms(image=img, bboxes=[gold_bbox], labels=[1])
        elif self.do_crop=="pred_boxes":
          pred_bbox = self.predicted_boxes[img_id]['Boxes']
          transformed_dict = self.transforms(image=img, bboxes=[pred_bbox], labels=[1])

        #visualize(img, pred_bboxes=[gold_bbox, pred_bbox], title=f"raw image {img_id}")
        transformed_img = transformed_dict['image'].float().div(255)
        transformed_bboxes = [round(x) for x in transformed_dict['bboxes'][0]]
        #visualize(transformed_img, [[0,0,0,0], transformed_bboxes], title=f"transformed img {img_id}")

        if self.do_crop is not None:
          transformed_bboxes_area = (transformed_bboxes[2]-transformed_bboxes[0]) * (transformed_bboxes[3]-transformed_bboxes[1])
          if transformed_bboxes_area > 0:
            transformed_img = crop_image(transformed_img, transformed_bboxes, p=self.p)
            transformed_img = F.interpolate(transformed_img.unsqueeze(0), size=(self.height, self.width), mode='bilinear', align_corners=False).squeeze(0)

        #visualize(transformed_img, title=f"cropped img {img_id}")

        target = {}
        target['boxes'] = torch.tensor(transformed_bboxes).unsqueeze(0)
        target['labels'] = torch.tensor(1).unsqueeze(0)
        target['true_labels'] = torch.tensor(img_label, dtype=torch.int64)
        target['image shape'] = {'height': img_height, 'width': img_width}
        target['Gold'] = gold_bbox
        return img_id, transformed_img, target

def collate_fn(batch):
  images_ids, images, targets = zip(*batch)
  return list(images_ids), list(images), list(targets)

### Evaluation

In [ ]:
def compute_IoU(boxA, boxB):
  x1_A, y1_A, x2_A, y2_A = boxA
  x1_B, y1_B, x2_B, y2_B = boxB

  x1 = max(x1_A, x1_B)
  y1 = max(y1_A, y1_B)
  x2 = min(x2_A, x2_B)
  y2 = min(y2_A, y2_B)

  if x2 <= x1 or y2 <= y1:
    return torch.tensor(0).to(device)

  intersection_area = (x2 - x1) * (y2 - y1)
  boxA_area = (x2_A - x1_A) * (y2_A - y1_A)
  boxB_area = (x2_B - x1_B) * (y2_B - y1_B)

  iou = intersection_area / float(boxA_area + boxB_area - intersection_area)
  return iou.float()

def center_within_gold(box, gold):
  x_center = (box[0] + box[2])/2
  y_center = (box[1] + box[3])/2
  x_min_gold, x_max_gold = gold[0], gold[2]
  y_min_gold, y_max_gold = gold[1], gold[3]
  if (x_min_gold<x_center<x_max_gold) and (y_min_gold<y_center<y_max_gold):
    return True
  else:
    return False

def compute_bbox_metrics(targets, predictions):
  tp = 0
  fp = 0
  fn = 0
  for i in range(len(predictions)):
    target_box = targets[i]['boxes'][0]
    pred_boxes = predictions[i]['boxes']
    if len(pred_boxes) > 0:
      for pred_box in pred_boxes:
        if center_within_gold(pred_box, target_box):
          tp += 1
        else:
          fp += 1
    else:
      fn += 1
  return tp, fp, fn

def evaluate_IoUs(targets, predictions):
  IoUs = []
  for i in range(len(predictions)):
    target_box = targets[i]['boxes'][0]
    pred_boxes = predictions[i]['boxes']
    if len(pred_boxes) > 0:
      for pred_box in pred_boxes:
        IoUs.append(compute_IoU(target_box, pred_box.squeeze(0)))
    else:
      IoUs.append(torch.tensor(0.0).to(device))
  return IoUs

# Retinanet

## Evaluation

In [ ]:
def evaluate_model(model, dataset_loader, width=224, height=224, return_bboxes=False):
    model.eval()
    all_IoUs = []
    TPs = 0
    FNs = 0
    FPs = 0
    precision = 0
    recall = 0
    all_bboxes= {}
    #mAP_metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox')
    with torch.inference_mode():
      for i, data in enumerate(dataset_loader, 0):
        images_ids, inputs, targets = data[0],\
                                      [image.to(device) for image in data[1]],\
                                      [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

        predictions = model(inputs)
        foreground_preds = filter_background(predictions)
        IoUs = evaluate_IoUs(targets, foreground_preds)
        all_IoUs += IoUs
        #mAP_metric.update(foreground_preds, targets)

        tp, fp, fn = compute_bbox_metrics(targets, foreground_preds)
        TPs += tp
        FPs += fp
        FNs += fn

        if return_bboxes:
          for i in range(len(targets)):
            fname = images_ids[i]
            target_bbox = targets[i]['Gold']
            bbox_transform = img_bbox_transform(target_height=targets[i]['image shape']['height'], target_width=targets[i]['image shape']['width'])
            pred_bbox = foreground_preds[i]['boxes'].squeeze(0)
            _, pred_bbox = bbox_transform(bbox=pred_bbox, current_width=width, current_height=height)
            all_bboxes[fname] = {"Boxes": pred_bbox, "Gold": target_bbox}

    mIoU = torch.stack(all_IoUs).mean()
    medIoU = torch.stack(all_IoUs).median()
    #mAP_dict = mAP_metric.compute()
    #mAP = mAP_dict['map']
    print(f"mIoU = {mIoU}, median IoU: {medIoU}") #, mAP = {round(mAP_dict['map'].item(),3)}, mAP_50 = {round(mAP_dict['map_50'].item(),3)}, mAP_75 = {round(mAP_dict['map_75'].item(),3)}")

    if (TPs + FPs) != 0:
      precision = TPs / (TPs + FPs)
    if (TPs + FNs) != 0:
      recall = TPs / (TPs + FNs)
    print(f"TP: {TPs}, FN: {FNs}, FP: {FPs}")
    print("Precision: {}\nRecall: {}".format(precision, recall))
    return None, all_bboxes

## Training

In [ ]:
def train_model(model, save_model_dir, num_epochs, optimizer, best_mAP=0, last_epoch=0, train_transform=None, eval_transform=None,
                num_worse_epochs=0, batch_size=10, patience=None, checkpoint_save_rate=5, lr_scheduler = None, train_scores=[], val_scores=[]):

    # Load train set
    train_data = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, img_transforms=train_transform)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Load validation set
    val_data = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, img_transforms=eval_transform)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    best_epoch = 0
    best_model = None

    os.makedirs(f'{save_model_dir}/checkpoints', exist_ok=True)
    if patience is not None:
      os.makedirs(f'{save_model_dir}/Final Models', exist_ok=True)

    print(f"Best mAP so far: {best_mAP}")

    for epoch in range(last_epoch+1, num_epochs+1):

      print(f'\nEpoch {epoch}:')
      start_time = time.time()

      ### Training Loop
      model.train()
      running_loss = 0.0
      total_step = len(train_loader)
      for i, data in enumerate(train_loader, 0):
          images_ids, inputs, targets = data[0],\
                                        [image.to(device) for image in data[1]],\
                                        [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward pass
          loss_dict = model(inputs, targets)

          # loss
          losses = sum(loss for loss in loss_dict.values())
          running_loss += losses

          # loss backward
          losses.backward()

          # optimize(update weights)
          optimizer.step()

      train_loss = running_loss/total_step
      print(f"Train Loss: {train_loss}, Total step: {total_step}")

      model.eval()
      ### Evaluation Loop (training data)
      print("Training evaluation:")
      train_mAP, _ = evaluate_model(model, train_loader, return_bboxes=False)
      train_scores.append(train_mAP)

      ### Evaluation Loop (validation data)
      print("Validation evaluation:")
      val_mAP, _ = evaluate_model(model, val_loader, return_bboxes=False)
      val_scores.append(val_mAP)

      # Early Stop
      #if val_mAP > best_mAP:
      #  best_mAP = val_mAP
      #  best_epoch = epoch
      #  best_model = copy.deepcopy(model)
      #  num_worse_epochs = 0
      #else:
      #  num_worse_epochs += 1
      #  if patience is not None:
      #    if num_worse_epochs >= patience:
      #      # save the best model and its optimizer
      #      if best_model is not None:
      #        torch.save({'model_state_dict': best_model.state_dict(),
      #                    'optimizer_state_dict': optimizer.state_dict(),
      #                    'best_mAP': best_mAP,
      #                    'epoch': best_epoch},
      #                    f'{save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')
      #        print(f'Best model is saved to: {save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')
      #        # save the last model and its optimizer
      #        torch.save({'model_state_dict': model.state_dict(),
      #                    'optimizer_state_dict': optimizer.state_dict(),
      #                    'best_mAP': best_mAP,
      #                    'epoch': epoch,
      #                    'num_worse_epochs': num_worse_epochs},
      #                    f'{save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')
      #        print(f'Last model is saved to: {save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')
      #      else:
      #        print("\nNo model to save!")
      #      print(f"No better results for last {patience} epochs!\nBest model was for epoch {best_epoch} with best mAP: {best_mAP}")
      #      break


      if (epoch) % checkpoint_save_rate == 0:
        random_state = random.getstate()
        np_random_state = np.random.get_state()
        torch_random_state = torch.get_rng_state()
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_mAP': best_mAP,
                    'train_scores': train_scores,
                    'val_scores': val_scores,
                    'epoch': epoch,
                    'num_worse_epochs': num_worse_epochs,
                    'random_state': random_state,
                    'np_random_state': np_random_state,
                    'torch_random_state': torch_random_state},
                    f'{save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth')
        print(f"\nCheckpoint is saved to: {save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth\n")

        if best_model is not None:
          torch.save({'model_state_dict': best_model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'best_mAP': best_mAP,
                      'epoch': best_epoch},
                      f'{save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
          print(f'Best model is saved to: {save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
          print(f"Best model was for epoch {best_epoch} with best mAP: {best_mAP}")

        else:
          print("\nNo Best model to save!")


      # Time
      end_time = time.time()
      elapsed_time = end_time - start_time
      print(f"Epoch time: {elapsed_time}")
      remaining_epochs = num_epochs - epoch - 1
      remaining_time = round(elapsed_time * remaining_epochs)
      hours, remainder = divmod(remaining_time, 3600)
      minutes, seconds = divmod(remainder, 60)
      print(f"Estimated time remaining: {hours}:{minutes}:{seconds}")

      if lr_scheduler is not None:
        lr_scheduler.step()


## Experiments

### Retinanet with COCO weights, fold0

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 99.2MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.464532494544983, Total step: 34
Training evaluation:
mIoU = 0.00792685616761446, median IoU: 0.0
TP: 21, FN: 0, FP: 1811
Precision: 0.01146288209606987
Recall: 1.0
Validation evaluation:
mIoU = 0.0079619400203228, median IoU: 0.0
TP: 1, FN: 0, FP: 203
Precision: 0.004901960784313725
Recall: 1.0
Epoch time: 205.35145664215088
Estimated time remaining: 2:44:17

Epoch 2:
Train Loss: 1.1460732221603394, Total step: 34
Training evaluation:
mIoU = 0.21936313807964325, median IoU: 0.13393372297286987
TP: 689, FN: 0, FP: 1143
Precision: 0.37609170305676853
Recall: 1.0
Validation evaluation:
mIoU = 0.23224937915802002, median IoU: 0.138303741812706
TP: 85, FN: 0, FP: 119
Precision: 0.4166666666666667
Recall: 1.0
Epoch time: 201.14824414253235
Estimated time remaining: 2:37:34

Epoch 3:
Train Loss: 0.8367591500282288, Total step: 34
Training evaluation:
mIoU = 0.4986112117767334, median IoU: 0.5287160873413086
TP: 1527, FN: 0, FP: 305
Precision: 0.83351

### Retinanet with COCO weights, fold1

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 138MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.466183066368103, Total step: 34
Training evaluation:
mIoU = 0.00693364767357707, median IoU: 0.0
TP: 13, FN: 0, FP: 1819
Precision: 0.007096069868995633
Recall: 1.0
Validation evaluation:
mIoU = 0.010101009160280228, median IoU: 0.0
TP: 1, FN: 0, FP: 203
Precision: 0.004901960784313725
Recall: 1.0
Epoch time: 205.37548780441284
Estimated time remaining: 2:44:18

Epoch 2:
Train Loss: 1.146626591682434, Total step: 34
Training evaluation:
mIoU = 0.2181405872106552, median IoU: 0.1312635838985443
TP: 696, FN: 0, FP: 1136
Precision: 0.3799126637554585
Recall: 1.0
Validation evaluation:
mIoU = 0.22473055124282837, median IoU: 0.158294215798378
TP: 82, FN: 0, FP: 122
Precision: 0.4019607843137255
Recall: 1.0
Epoch time: 201.16140246391296
Estimated time remaining: 2:37:35

Epoch 3:
Train Loss: 0.8365169167518616, Total step: 34
Training evaluation:
mIoU = 0.5012315511703491, median IoU: 0.5265228748321533
TP: 1530, FN: 0, FP: 302
Precision: 0.835152

### Retinanet with COCO weights, fold2

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 135MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.4653894901275635, Total step: 34
Training evaluation:
mIoU = 0.007118549197912216, median IoU: 0.0
TP: 18, FN: 0, FP: 1814
Precision: 0.009825327510917031
Recall: 1.0
Validation evaluation:
mIoU = 0.005204278044402599, median IoU: 0.0
TP: 0, FN: 0, FP: 204
Precision: 0.0
Recall: 0
Epoch time: 204.16729855537415
Estimated time remaining: 2:43:20

Epoch 2:
Train Loss: 1.1482117176055908, Total step: 34
Training evaluation:
mIoU = 0.2181018739938736, median IoU: 0.13356532156467438
TP: 701, FN: 0, FP: 1131
Precision: 0.3826419213973799
Recall: 1.0
Validation evaluation:
mIoU = 0.21609070897102356, median IoU: 0.12238644063472748
TP: 69, FN: 0, FP: 135
Precision: 0.3382352941176471
Recall: 1.0
Epoch time: 198.81996202468872
Estimated time remaining: 2:35:45

Epoch 3:
Train Loss: 0.8395388722419739, Total step: 34
Training evaluation:
mIoU = 0.5042182803153992, median IoU: 0.5382752418518066
TP: 1542, FN: 0, FP: 290
Precision: 0.8417030567685589
Re

### Retinanet with COCO weights, fold3

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:09<00:00, 16.9MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.4632105827331543, Total step: 34
Training evaluation:
mIoU = 0.00806721393018961, median IoU: 0.0
TP: 20, FN: 0, FP: 1812
Precision: 0.010917030567685589
Recall: 1.0
Validation evaluation:
mIoU = 0.007824249565601349, median IoU: 0.0
TP: 1, FN: 0, FP: 203
Precision: 0.004901960784313725
Recall: 1.0
Epoch time: 204.53067088127136
Estimated time remaining: 2:43:37

Epoch 2:
Train Loss: 1.1433358192443848, Total step: 34
Training evaluation:
mIoU = 0.2274944931268692, median IoU: 0.1334282010793686
TP: 700, FN: 0, FP: 1132
Precision: 0.38209606986899564
Recall: 1.0
Validation evaluation:
mIoU = 0.22768232226371765, median IoU: 0.1506318897008896
TP: 85, FN: 0, FP: 119
Precision: 0.4166666666666667
Recall: 1.0
Epoch time: 201.336905002594
Estimated time remaining: 2:37:43

Epoch 3:
Train Loss: 0.8354271650314331, Total step: 34
Training evaluation:
mIoU = 0.5017008185386658, median IoU: 0.531846821308136
TP: 1529, FN: 0, FP: 303
Precision: 0.83460

### Retinanet with COCO weights, fold4

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 116MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.466421127319336, Total step: 34
Training evaluation:
mIoU = 0.007386784069240093, median IoU: 0.0
TP: 20, FN: 0, FP: 1812
Precision: 0.010917030567685589
Recall: 1.0
Validation evaluation:
mIoU = 0.004110545851290226, median IoU: 0.0
TP: 0, FN: 0, FP: 204
Precision: 0.0
Recall: 0
Epoch time: 217.16205716133118
Estimated time remaining: 2:53:44

Epoch 2:
Train Loss: 1.1456290483474731, Total step: 34
Training evaluation:
mIoU = 0.210371732711792, median IoU: 0.12856678664684296
TP: 664, FN: 0, FP: 1168
Precision: 0.3624454148471616
Recall: 1.0
Validation evaluation:
mIoU = 0.21119260787963867, median IoU: 0.10400800406932831
TP: 74, FN: 0, FP: 130
Precision: 0.3627450980392157
Recall: 1.0
Epoch time: 216.00250816345215
Estimated time remaining: 2:49:12

Epoch 3:
Train Loss: 0.8365991711616516, Total step: 34
Training evaluation:
mIoU = 0.498136043548584, median IoU: 0.5316162705421448
TP: 1520, FN: 0, FP: 312
Precision: 0.8296943231441049
Recal

### Retinanet with COCO weights, fold5

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 86.4MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.4644914865493774, Total step: 34
Training evaluation:
mIoU = 0.008374392054975033, median IoU: 0.0
TP: 18, FN: 0, FP: 1814
Precision: 0.009825327510917031
Recall: 1.0
Validation evaluation:
mIoU = 0.006230880506336689, median IoU: 0.0
TP: 1, FN: 0, FP: 203
Precision: 0.004901960784313725
Recall: 1.0
Epoch time: 203.6903853416443
Estimated time remaining: 2:42:57

Epoch 2:
Train Loss: 1.148182988166809, Total step: 34
Training evaluation:
mIoU = 0.21543285250663757, median IoU: 0.12329741567373276
TP: 663, FN: 0, FP: 1169
Precision: 0.3618995633187773
Recall: 1.0
Validation evaluation:
mIoU = 0.21657462418079376, median IoU: 0.13737884163856506
TP: 78, FN: 0, FP: 126
Precision: 0.38235294117647056
Recall: 1.0
Epoch time: 204.55470156669617
Estimated time remaining: 2:40:14

Epoch 3:
Train Loss: 0.8418301939964294, Total step: 34
Training evaluation:
mIoU = 0.4978644549846649, median IoU: 0.5234065651893616
TP: 1524, FN: 0, FP: 308
Precision: 0.

### Retinanet with COCO weights, fold6

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 151MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.463950276374817, Total step: 34
Training evaluation:
mIoU = 0.007019410375505686, median IoU: 0.0
TP: 8, FN: 0, FP: 1825
Precision: 0.00436442989634479
Recall: 1.0
Validation evaluation:
mIoU = 0.00790954940021038, median IoU: 0.0
TP: 2, FN: 0, FP: 201
Precision: 0.009852216748768473
Recall: 1.0
Epoch time: 201.6396608352661
Estimated time remaining: 2:41:19

Epoch 2:
Train Loss: 1.1457786560058594, Total step: 34
Training evaluation:
mIoU = 0.21370600163936615, median IoU: 0.1285618096590042
TP: 669, FN: 0, FP: 1164
Precision: 0.3649754500818331
Recall: 1.0
Validation evaluation:
mIoU = 0.23026564717292786, median IoU: 0.15138204395771027
TP: 79, FN: 0, FP: 124
Precision: 0.3891625615763547
Recall: 1.0
Epoch time: 199.2471466064453
Estimated time remaining: 2:36:5

Epoch 3:
Train Loss: 0.8355892300605774, Total step: 34
Training evaluation:
mIoU = 0.4976123571395874, median IoU: 0.5284537672996521
TP: 1524, FN: 0, FP: 309
Precision: 0.8314238

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 100
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: RN, COCO/checkpoints/checkpoint_epoch_25.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 125MB/s]


Best mAP so far: 0

Epoch 26:
Train Loss: 0.3734554350376129, Total step: 34
Training evaluation:
mIoU = 0.7408830523490906, median IoU: 0.7710378766059875
TP: 1807, FN: 0, FP: 26
Precision: 0.9858156028368794
Recall: 1.0
Validation evaluation:
mIoU = 0.6927283406257629, median IoU: 0.7285062670707703
TP: 195, FN: 0, FP: 8
Precision: 0.9605911330049262
Recall: 1.0
Epoch time: 209.12563180923462
Estimated time remaining: 4:14:26

Epoch 27:
Train Loss: 0.36907723546028137, Total step: 34
Training evaluation:
mIoU = 0.7459397912025452, median IoU: 0.7785996794700623
TP: 1809, FN: 0, FP: 24
Precision: 0.9869067103109657
Recall: 1.0
Validation evaluation:
mIoU = 0.6919940114021301, median IoU: 0.7216110229492188
TP: 195, FN: 0, FP: 8
Precision: 0.9605911330049262
Recall: 1.0
Epoch time: 207.83862352371216
Estimated time remaining: 4:9:24

Epoch 28:
Train Loss: 0.36440980434417725, Total step: 34
Training evaluation:
mIoU = 0.7456113696098328, median IoU: 0.7773049473762512
TP: 1807, FN: 0, 

### Retinanet with COCO weights, fold7

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 93.1MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.463027000427246, Total step: 34
Training evaluation:
mIoU = 0.007886097766458988, median IoU: 0.0
TP: 22, FN: 0, FP: 1811
Precision: 0.012002182214948172
Recall: 1.0
Validation evaluation:
mIoU = 0.006478908006101847, median IoU: 0.0
TP: 1, FN: 0, FP: 202
Precision: 0.0049261083743842365
Recall: 1.0
Epoch time: 206.65855360031128
Estimated time remaining: 2:45:20

Epoch 2:
Train Loss: 1.1433955430984497, Total step: 34
Training evaluation:
mIoU = 0.2327825129032135, median IoU: 0.146190345287323
TP: 752, FN: 0, FP: 1081
Precision: 0.41025641025641024
Recall: 1.0
Validation evaluation:
mIoU = 0.2198282927274704, median IoU: 0.12034723162651062
TP: 74, FN: 0, FP: 129
Precision: 0.3645320197044335
Recall: 1.0
Epoch time: 203.33820819854736
Estimated time remaining: 2:39:17

Epoch 3:
Train Loss: 0.8366847038269043, Total step: 34
Training evaluation:
mIoU = 0.4943554103374481, median IoU: 0.5137047171592712
TP: 1525, FN: 0, FP: 308
Precision: 0.83

### Retinanet with COCO weights, fold8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 145MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.4639872312545776, Total step: 34
Training evaluation:
mIoU = 0.00728031899780035, median IoU: 0.0
TP: 21, FN: 0, FP: 1812
Precision: 0.011456628477905073
Recall: 1.0
Validation evaluation:
mIoU = 0.004372128751128912, median IoU: 0.0
TP: 0, FN: 0, FP: 203
Precision: 0.0
Recall: 0
Epoch time: 212.87005424499512
Estimated time remaining: 2:50:18

Epoch 2:
Train Loss: 1.1420897245407104, Total step: 34
Training evaluation:
mIoU = 0.2351670265197754, median IoU: 0.14245633780956268
TP: 749, FN: 0, FP: 1084
Precision: 0.408619749045281
Recall: 1.0
Validation evaluation:
mIoU = 0.2411143183708191, median IoU: 0.12656335532665253
TP: 82, FN: 0, FP: 121
Precision: 0.4039408866995074
Recall: 1.0
Epoch time: 211.82416772842407
Estimated time remaining: 2:45:56

Epoch 3:
Train Loss: 0.835761308670044, Total step: 34
Training evaluation:
mIoU = 0.5025224685668945, median IoU: 0.5281828045845032
TP: 1541, FN: 0, FP: 292
Precision: 0.8406983087834151
Recall

### Retinanet with COCO weights, fold9

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 50
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 135MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.46159827709198, Total step: 34
Training evaluation:
mIoU = 0.00923166424036026, median IoU: 0.0
TP: 25, FN: 0, FP: 1808
Precision: 0.013638843426077468
Recall: 1.0
Validation evaluation:
mIoU = 0.008065182715654373, median IoU: 0.0
TP: 1, FN: 0, FP: 202
Precision: 0.0049261083743842365
Recall: 1.0
Epoch time: 212.2766351699829
Estimated time remaining: 2:49:49

Epoch 2:
Train Loss: 1.1395941972732544, Total step: 34
Training evaluation:
mIoU = 0.22562995553016663, median IoU: 0.13872307538986206
TP: 715, FN: 0, FP: 1118
Precision: 0.3900709219858156
Recall: 1.0
Validation evaluation:
mIoU = 0.2185061126947403, median IoU: 0.12120386958122253
TP: 65, FN: 0, FP: 138
Precision: 0.32019704433497537
Recall: 1.0
Epoch time: 206.81682538986206
Estimated time remaining: 2:42:0

Epoch 3:
Train Loss: 0.83576500415802, Total step: 34
Training evaluation:
mIoU = 0.5028902888298035, median IoU: 0.5442845821380615
TP: 1539, FN: 0, FP: 294
Precision: 0.83960

# GBCRet

## Evaluation

In [ ]:
def evaluate_model(model, dataloader, return_bboxes=False):
  model.eval()
  all_IoUs = []
  #mAP_metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox')
  y_pred = []
  y_true = []
  fnames = []
  TPs = 0
  FNs = 0
  FPs = 0
  precision = 0
  recall = 0
  all_bboxes= {}
  with torch.inference_mode():
    for i, data in enumerate(dataloader, 0):
      images_ids, inputs, targets = data[0],\
                                    [image.to(device) for image in data[1]],\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      pred_cls, pred_bboxes = model(inputs, targets)

      y_true += [x['true_labels'].item() for x in targets]
      y_pred += pred_cls.tolist()
      fnames += images_ids

      IoUs = evaluate_IoUs(targets, pred_bboxes)
      all_IoUs += IoUs

      tp, fp, fn = compute_bbox_metrics(targets, pred_bboxes)
      TPs += tp
      FPs += fp
      FNs += fn

      #mAP_metric.update(pred_bboxes, targets)

      if return_bboxes:
        for i in range(len(targets)):
          fname = images_ids[i]
          target_bbox = targets[i]['boxes']
          pred_bbox = pred_bboxes[i]['boxes']
          all_bboxes[fname] = {"Boxes": pred_bbox, "Gold": target_bbox}

  mIoU = torch.stack(all_IoUs).mean()
  medIoU = torch.stack(all_IoUs).median()
  #mAP_dict = mAP_metric.compute()
  #mAP = mAP_dict['map']
  print(f"mIoU = {mIoU}, median IoU: {medIoU}") #, mAP = {round(mAP_dict['map'].item(),3)}, mAP_50 = {round(mAP_dict['map_50'].item(),3)}, mAP_75 = {round(mAP_dict['map_75'].item(),3)}")

  if (TPs + FPs) != 0:
    precision = TPs / (TPs + FPs)
  if (TPs + FNs) != 0:
    recall = TPs / (TPs + FNs)
  print(f"TP: {TPs}, FN: {FNs}, FP: {FPs}")
  print("Precision: {}\nRecall: {}".format(precision, recall))

  acc = accuracy_score(y_true, y_pred)
  cfm = confusion_matrix(y_true, y_pred)
  spec = (cfm[0][0] + cfm[0][1] + cfm[1][0] + cfm[1][1])/(np.sum(cfm[0]) + np.sum(cfm[1]))
  sens = cfm[2][2]/np.sum(cfm[2])
  n_nn_acc = (cfm[0,0] + np.sum(cfm[1:,1:]))/np.sum(cfm)
  b_m_acc = (cfm[1,1] + cfm[2,2])/np.sum(cfm[1:,1:])
  print('Acc: {:.4f} Spec: {:.4f} Sens: {:.4f}\nNormal vs Not Noraml Acc: {:.4f} Benign vs Malignant Acc: {:.4f}'.format(acc, spec, sens, n_nn_acc, b_m_acc))

  #return acc, round(mAP_dict['map'].item(),3), cfm, y_true, y_pred, fnames, all_bboxes
  return acc, None, cfm, y_true, y_pred, fnames, all_bboxes

## Model

### Attention

In [ ]:
class FeatureMapAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.multiheadattn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

  def forward(self, feature_map):
    # feature map shape: [batch_size, 256, H, W]
    # Reshape to: [batch_size, H*W, 256]
    x = feature_map.reshape(feature_map.size(0), -1, 256)
    # Apply Attention
    attn_output, _ = self.multiheadattn(x,x,x)
    # Reshape back to original shape: [batch_size, 256, H, W]
    return attn_output.reshape(*feature_map.shape)

class VectorAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.multiheadattn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

  def forward(self, x):
    # x shape: [batch_size, 256 + 2048]
    # Reshape to: [batch_size, 1, 256 + 2048]
    x = x.unsqueeze(1)
    # Apply Attention
    attn_output, _ = self.multiheadattn(x, x, x)
    # Remove the extra dimension: [batch_size, 256 + 2048]
    return attn_output.squeeze(1)


### Main model

In [ ]:
class GBCRet_combined(nn.Module):
  def __init__(self, combined="None", continue_training=False, criterion=None, gbcnet_weight_path=None, retinanet_weight_path=None,
               do_hierarchical_cls=False, retinanet_backbone="resnet50-coco", gold_curriculum=False, p=0.15, width=224, height=224,
               attn_num_heads=4, no_attn=True, train_gold_bbox=True, min_size=800, max_size=1333, dropout=0.4, detections_per_img=1,
               regression_loss=None, rn_trainable_layers=None, gbcnet_va_curriculum=False, load_pretrained_retinanet=False,
               nnn_fc_layers="1", bm_fc_layers="1",
               loss_weights = {"n_nn_loss_weight": 1.0, "b_m_loss_weight": 1.0, "classification_loss_weight": 1.0, "retinanet_loss_weight": 1.0}):
    super(GBCRet_combined, self).__init__()
    # Settings
    self.loss_weights=loss_weights
    self.load_pretrained_retinanet=load_pretrained_retinanet
    self.gbcnet_va_curriculum=gbcnet_va_curriculum
    self.rn_trainable_layers=rn_trainable_layers
    self.retinanet_backbone=retinanet_backbone
    self.combined=combined
    self.no_attn = no_attn
    self.do_hierarchical_cls=do_hierarchical_cls
    self.continue_training=continue_training
    self.detections_per_img=detections_per_img
    self.p = p
    self.width = width
    self.height = height
    self.min_size = min_size
    self.max_size = max_size
    self.criterion = criterion
    self.train_gold_bbox = train_gold_bbox
    self.gold_curriculum = gold_curriculum
    self.regression_loss = regression_loss
    self.current_epoch = -1
    if isinstance(dropout, dict):
      n_nn_dropout=dropout['n_nn_dropout']
      b_m_dropout=dropout['b_m_dropout']
      dropout=0
      print(f"n_nn_dropout: {n_nn_dropout}, b_m_dropout: {b_m_dropout}")
    else:
      n_nn_dropout=dropout
      b_m_dropout=dropout
    # Status Flags
    self.gbcnet_va_curriculum_flag=False
    self.combined_status_flag=False
    self.no_attn_status_flag=False
    self.train_gold_box_status_flag=False
    self.gold_curriculum_status_flag=False
    self.num_gold_boxes_status_flag=False
    self.hierarchical_classification_status_flag=False
    # Loading models
    self.retinanet = self.load_retinanet(retinanet_weight_path)
    self.gbcnet = self.load_gbcnet(gbcnet_weight_path)
    # Hooks
    self.retinanet_features = None
    self.gbcnet_features = None
    self.resnet_layer1_features = None
    self.resnet_layer2_features = None
    self.resnet_layer3_features = None
    self.resnet_layer4_features = None
    self.gbcnet.net.layer4.register_forward_hook(self._get_gbcnet_features_hook)
    self.retinanet.backbone.register_forward_hook(self._get_retinanet_features_hook)
    self.retinanet.backbone.body.layer1.register_forward_hook(self._get_resnet_layer1_features_hook)
    self.retinanet.backbone.body.layer2.register_forward_hook(self._get_resnet_layer2_features_hook)
    self.retinanet.backbone.body.layer3.register_forward_hook(self._get_resnet_layer3_features_hook)
    self.retinanet.backbone.body.layer4.register_forward_hook(self._get_resnet_layer4_features_hook)
    # Layers
    self.avgpool = nn.AdaptiveAvgPool2d(1)
    self.feature_map_attn = FeatureMapAttention(embed_dim=256, num_heads=attn_num_heads)
    self.combined_attn_layer = VectorAttention(embed_dim= 2048 + 256, num_heads=attn_num_heads)

    self.fc = nn.Sequential(
                  nn.Linear((2048 + 256), 256),
                  nn.ReLU(inplace=True),
                  nn.Dropout(dropout),
                  nn.Linear(256, 3)
                )
    self.FPN_linear = nn.Linear(5*256, 256)
    self.Layer2_linear = nn.Linear(512, 256)
    self.Layer3_linear = nn.Linear(1024, 256)
    self.Layer4_linear = nn.Linear(2048, 256)
    self.compress_combined_featuers_linear = nn.Linear((2048 + 256), 2048)

    self.fc2 = nn.Sequential(
                  nn.Linear((5*256), 256),
                  nn.ReLU(inplace=True),
                  nn.Dropout(dropout),
                  nn.Linear(256, 3)
                )
    self.meta_learner = nn.Linear(6,3)

    if nnn_fc_layers=="1":
      self.normal_notnormal_fc = nn.Sequential(
                                            nn.Linear(2048, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(n_nn_dropout),
                                            nn.Linear(256,2)
                                          )
    elif nnn_fc_layers=="3":
      self.normal_notnormal_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(n_nn_dropout),
                                            nn.Linear(512,2)
                                          )
    if bm_fc_layers == "1":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(256,2)
                                          )
    elif bm_fc_layers == "2":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(dropout),
                                            nn.Linear(512, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(256, 2)
                                          )
    elif bm_fc_layers == "3":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(512,2)
                                          )
    elif bm_fc_layers == "4":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 1024),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(1024,2)
                                          )

  def load_retinanet(self, retinanet_weight_path):
    if self.retinanet_backbone=="resnet50-gbusv":
      # No weights -> by default trainable_layers = 5 - https://github.com/pytorch/vision/blob/b1123cfd543d35d282de9bb28067e48ebec18afe/torchvision/models/detection/retinanet.py#L879
      retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img = self.detections_per_img, score_thresh=0.05,
                                            nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size).to(device)
      if not self.continue_training:
        retinanet.load_state_dict(torch.load(retinanet_weight_path, map_location=device))
        print(f"Retinanet's GBUSV pre-trained weights are loaded from: {retinanet_weight_path}")
      return retinanet
    elif self.retinanet_backbone=="resnet50-coco":
      if self.rn_trainable_layers == None:
        # Defined weights -> by default trainable layers = 3 - https://github.com/pytorch/vision/blob/b1123cfd543d35d282de9bb28067e48ebec18afe/torchvision/models/detection/retinanet.py#L879
        retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=self.detections_per_img, score_thresh=0.05,
                                              nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size).to(device)
      else:
        print(f"Retinanet's Trainable Backbone Layers: {self.rn_trainable_layers}")
        retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=self.detections_per_img, score_thresh=0.05,
                                              nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size,
                                              trainable_backbone_layers=self.rn_trainable_layers).to(device)
      print("Retinanet's COCO pre-trained weights are loaded.")
      num_classes=2
      num_anchors = retinanet.head.classification_head.num_anchors
      in_channels = retinanet.head.classification_head.cls_logits.in_channels
      new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
      torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
      torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
      retinanet.head.classification_head.cls_logits = new_cls_logits
      retinanet.head.classification_head.num_classes = num_classes
      if self.load_pretrained_retinanet:
        checkpoint = torch.load(retinanet_weight_path, map_location=device)
        retinanet.load_state_dict(checkpoint['model_state_dict'])
        print(f"Retinanet's {checkpoint['epoch']} epoch Pretrained Weights from: {retinanet_weight_path}")
      return retinanet
    elif self.retinanet_backbone=="resnet34" or self.retinanet_backbone=="resnet18":
      # Create the backbone using ResNet34 with FPN
      backbone = resnet_fpn_backbone(backbone_name=self.retinanet_backbone, trainable_layers=3, weights="IMAGENET1K_V1").to(device)
      # Create the RetinaNet model
      retinanet = RetinaNet(backbone, num_classes=2, detections_per_img=self.detections_per_img, score_thresh=0.05, nms_thresh=0.5,
                            min_size=self.min_size, max_size=self.max_size).to(device)
      if self.regression_loss == 'giou':
        retinanet.head.regression_head._loss_type = "giou"
      return retinanet

  def load_gbcnet(self, gbcnet_weight_path):
    gbcnet = GbcNet(num_cls=3, pretrain=False, att_mode= "1")
    if not self.continue_training:
      gbcnet.load_state_dict(torch.load(gbcnet_weight_path, map_location=device))
      print(f"GBCNet's pretrained weights are loaded from: {gbcnet_weight_path}")
    gbcnet.net = gbcnet.net.float().to(device)
    return gbcnet

  def _get_retinanet_features_hook(self, module, input, output):
    self.retinanet_features = output

  def _get_gbcnet_features_hook(self, module, input, output):
    self.gbcnet_features = output

  def _get_resnet_layer1_features_hook(self, module, input, output):
    self.resnet_layer1_features = output

  def _get_resnet_layer2_features_hook(self, module, input, output):
    self.resnet_layer2_features = output

  def _get_resnet_layer3_features_hook(self, module, input, output):
    self.resnet_layer3_features = output

  def _get_resnet_layer4_features_hook(self, module, input, output):
    self.resnet_layer4_features = output

  def forward(self, inputs, targets, epoch=None):
    if self.training:
      retinanet_loss_dict = self.retinanet(inputs, targets)
      retinanet_loss = sum(loss for loss in retinanet_loss_dict.values())
      retinanet_features = self.retinanet_features
      resnet_layer1_features = self.resnet_layer1_features
      resnet_layer2_features = self.resnet_layer2_features
      resnet_layer3_features = self.resnet_layer3_features
      resnet_layer4_features = self.resnet_layer4_features

      if self.gold_curriculum==False:
        self.gold_curriculum_status_flag = print_status("Gold Curriculum Deactivated.", self.gold_curriculum_status_flag)
        if self.train_gold_bbox:
          self.train_gold_box_status_flag = print_status("Training only with gold boxes.", self.train_gold_box_status_flag)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, targets, targets, epoch)
        else:
          self.train_gold_box_status_flag = print_status("Training only with predicted boxes.", self.train_gold_box_status_flag)
          pred_bboxes = self.predict_bboxes(inputs)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, pred_bboxes, targets)

      else:
        self.gold_curriculum_status_flag = print_status("Gold Curriculum Activated.", self.gold_curriculum_status_flag)
        if self.current_epoch != epoch:
          self.num_gold_boxes_status_flag=False
        self.current_epoch=epoch
        if gold_box_proportion(epoch) == 1:
          self.num_gold_boxes_status_flag = print_status("All Gold Boxes are using.", self.num_gold_boxes_status_flag)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, targets, targets)
        else:
          num_gold_boxes = int(gold_box_proportion(epoch)*len(targets))
          self.num_gold_boxes_status_flag = print_status(f"{num_gold_boxes} Gold Boxes, and {len(targets)-num_gold_boxes} Predicted Boxes are using.", self.num_gold_boxes_status_flag)
          pred_bboxes = self.predict_bboxes(inputs)
          selected_bboxes = targets[:num_gold_boxes]
          selected_bboxes += pred_bboxes[num_gold_boxes:]
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, selected_bboxes, targets)

      gbcnet_cls_logits = self.gbcnet(gbcnet_inputs)
      gbcnet_features = self.gbcnet_features

      if self.combined=='resnet-fpn':
        self.combined_status_flag = print_status("Combining features of Retinanet and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(retinanet_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer1":
        self.combined_status_flag = print_status("Combining ResNet layer1 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer1_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer2":
        self.combined_status_flag = print_status("Combining ResNet layer2 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer2_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer3":
        self.combined_status_flag = print_status("Combining ResNet layer3 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer3_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer4":
        self.combined_status_flag = print_status("Combining ResNet layer4 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer4_features, gbcnet_features)
        if not self.do_hierarchical_cls:
          cls_output = self.fc(combined_features)
          classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
        else:
          combined_features = self.compress_combined_featuers_linear(combined_features)
          classification_loss = self.hierarchical_cls(combined_features, true_labels)
      elif self.combined=="classification":
        self.combined_status_flag = print_status("Combining classifications of Retinanet and GBCNet backbones.", self.combined_status_flag)
        retinanet_features = self.concat_feature_maps(retinanet_features)
        retinanet_cls_logits = self.fc2(retinanet_features)
        combined_cls_logits = self.meta_learner(torch.cat((gbcnet_cls_logits, retinanet_cls_logits), dim=1))
        classification_loss = self.criterion(combined_cls_logits.to(device), true_labels.to(device))
      elif self.combined=='None':
        if not self.do_hierarchical_cls:
          self.combined_status_flag = print_status("Classification only using GBCNet.", self.combined_status_flag)
          classification_loss = self.criterion(gbcnet_cls_logits.to(device), true_labels.to(device))
        else:
          self.hierarchical_classification_status_flag = print_status("Hierarchical Classification is Activated.", self.hierarchical_classification_status_flag)
          gbcnet_features = self.avgpool(gbcnet_features)
          gbcnet_features = gbcnet_features.view(gbcnet_features.size(0), -1)
          classification_loss = self.hierarchical_cls(gbcnet_features, true_labels)

      total_loss = self.loss_weights["retinanet_loss_weight"] * retinanet_loss + self.loss_weights["classification_loss_weight"] * classification_loss
      return total_loss

    else:
      pred_bboxes = self.predict_bboxes(inputs)
      retinanet_features = self.retinanet_features
      resnet_layer1_features = self.resnet_layer1_features
      resnet_layer2_features = self.resnet_layer2_features
      resnet_layer3_features = self.resnet_layer3_features
      resnet_layer4_features = self.resnet_layer4_features
      gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, pred_bboxes, targets)
      gbcnet_cls_logits = self.gbcnet(gbcnet_inputs)
      gbcnet_features = self.gbcnet_features

      if self.combined=='resnet-fpn':
        combined_features = self.concat_feature_maps(retinanet_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer1":
        combined_features = self.concat_feature_maps(resnet_layer1_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer2":
        combined_features = self.concat_feature_maps(resnet_layer2_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer3":
        combined_features = self.concat_feature_maps(resnet_layer3_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer4":
        combined_features = self.concat_feature_maps(resnet_layer4_features, gbcnet_features)
        if not self.do_hierarchical_cls:
          cls_output = self.fc(combined_features)
        else:
          combined_features = self.compress_combined_featuers_linear(combined_features)
          final_h_preds = self.hierarchical_cls(combined_features, true_labels)
      elif self.combined=='classification':
        retinanet_features = self.concat_feature_maps(retinanet_features)
        retinanet_cls_logits = self.fc2(retinanet_features)
        combined_cls_logits = self.meta_learner(torch.cat((gbcnet_cls_logits, retinanet_cls_logits), dim=1))
        cls_output = combined_cls_logits
      elif self.combined=='None':
        if not self.do_hierarchical_cls:
          cls_output = gbcnet_cls_logits
        else:
          gbcnet_features = self.avgpool(gbcnet_features)
          gbcnet_features = gbcnet_features.view(gbcnet_features.size(0), -1)
          final_h_preds = self.hierarchical_cls(gbcnet_features, true_labels)

      if not self.do_hierarchical_cls:
        _, cls_pred = torch.max(cls_output, dim=1)
      else:
        cls_pred = final_h_preds

      return cls_pred, pred_bboxes

  def hierarchical_cls(self, features, true_labels):
    if self.training:
      self.hierarchical_classification_status_flag = print_status("Hierarchical Classification is Activated.", self.hierarchical_classification_status_flag)

      # Normal vs Non-Normal classification
      n_nn_output = self.normal_notnormal_fc(features)
      n_nn_true_labels = torch.tensor([0 if x == 0 else 1 for x in true_labels]) # Normal-> 0, Non-Normal -> 1

      if isinstance(self.criterion, nn.BCEWithLogitsLoss):
        n_nn_true_labels = one_hot(n_nn_true_labels, num_classes=2).float()
      n_nn_loss = self.criterion(n_nn_output.to(device), n_nn_true_labels.to(device))

      not_normal_indices = (true_labels != 0)
      if not (not_normal_indices==False).all():
        # Benign vs Malignant classification
        b_m_output = self.benign_malignant_fc(features[not_normal_indices])
        b_m_true_labels = true_labels[not_normal_indices]
        b_m_true_labels = torch.tensor([0 if x==1 else 1 for x in b_m_true_labels]) # Benign -> 0, Malignant -> 1

        if isinstance(self.criterion, nn.BCEWithLogitsLoss):
          b_m_true_labels = one_hot(b_m_true_labels, num_classes=2).float()
        b_m_loss = self.criterion(b_m_output.to(device), b_m_true_labels.to(device))
      else:
        b_m_loss = 0
      loss = self.loss_weights["n_nn_loss_weight"] * n_nn_loss + self.loss_weights["b_m_loss_weight"] * b_m_loss
      return loss
    else:
      n_nn_output = self.normal_notnormal_fc(features)
      _, n_nn_cls_pred = torch.max(n_nn_output, dim=1)
      final_h_preds = torch.zeros_like(n_nn_cls_pred)
      not_normal_indices = (n_nn_cls_pred != 0)
      if not (not_normal_indices==False).all():
        b_m_output = self.benign_malignant_fc(features[not_normal_indices])
        _, b_m_cls_pred = torch.max(b_m_output, dim=1)
        b_m_cls_pred += 1
        final_h_preds[not_normal_indices] = b_m_cls_pred
      return final_h_preds


  def predict_bboxes(self, inputs):
    self.retinanet.eval()
    with torch.inference_mode():
      predictions = self.retinanet(inputs)
      foreground_preds = filter_background(predictions)
    if self.training:
      self.retinanet.train()
    return foreground_preds

  def va_blur(self, img, epoch):
    self.gbcnet_va_curriculum_flag = print_status("GBCNet VA Curriculum Activated.", self.gbcnet_va_curriculum_flag)
    if epoch <= 10:
      gaussian_blur = GaussianBlur((17,17), 8)
      img = gaussian_blur(img)
    elif epoch > 10 and epoch <= 15:
      gaussian_blur = GaussianBlur((9,9), 4)
      img = gaussian_blur(img)
    elif epoch > 15 and epoch <= 20:
      gaussian_blur = GaussianBlur((5,5), 2)
      img = gaussian_blur(img)
    elif epoch > 20 and epoch <= 25:
      gaussian_blur = GaussianBlur((3,3), 1)
      img = gaussian_blur(img)
    elif epoch > 25 and epoch <= 30:
      gaussian_blur = GaussianBlur((1,1), 0.5)
      img = gaussian_blur(img)
    else:
      return img
    return img

  def prepare_gbcnet_inputs(self, inputs, predictions, targets, epoch=None):
    true_labels = []
    gbcnet_inputs = []
    for i in range(len(inputs)):
      img = inputs[i]
      if (self.gbcnet_va_curriculum) and (epoch is not None):
        img = self.va_blur(img, epoch)
      true_labels.append(targets[i]['true_labels'])
      pred_bboxes = predictions[i]['boxes']
      #visualize(img)
      if len(pred_bboxes) > 0:
        for pred_bbox in pred_bboxes:
          pred_bbox_area = (int(pred_bbox[2])-int(pred_bbox[0])) * (int(pred_bbox[3])-int(pred_bbox[1]))
          if pred_bbox_area > 0:
            img = crop_image(img, pred_bbox, self.p)
            img = F.interpolate(img.unsqueeze(0), size=(self.height, self.width), mode='bilinear', align_corners=False).squeeze(0)
      gbcnet_inputs.append(img)
    return torch.stack(gbcnet_inputs), torch.stack(true_labels)

  def concat_feature_maps(self, rn_fms, gb_fm=None):
    if self.combined=='resnet-fpn':
      rn_fms = list(rn_fms.values())
      if not self.no_attn:
        self.no_attn_status_flag = print_status("Self attention on Retinanet's feature maps are applied.", self.no_attn_status_flag)
        rn_fms = [self.feature_map_attn(x) for x in rn_fms]
      rn_fms = [self.avgpool(x) for x in rn_fms]
      rn_fms = [x.view(x.size(0), -1) for x in rn_fms]
      rn_rms = torch.stack(rn_fms)
      rn_fms = torch.cat(rn_fms, dim=-1)
      rn_fms = self.FPN_linear(rn_fms)
      gb_fm = self.avgpool(gb_fm)
      gb_fm = gb_fm.view(gb_fm.size(0), -1)
      concatenated_features = torch.cat([rn_fms, gb_fm], dim = 1)
    elif self.combined in ["resnet_layer1", "resnet_layer2", "resnet_layer3", "resnet_layer4"]:
      rn_fms = self.avgpool(rn_fms)
      rn_fms = rn_fms.reshape(rn_fms.size(0), -1)
      if self.combined=="resnet_layer2":
        rn_fms = self.Layer2_linear(rn_fms)
      elif self.combined=="resnet_layer3":
        rn_fms = self.Layer3_linear(rn_fms)
      elif self.combined=="resnet_layer4":
        rn_fms = self.Layer4_linear(rn_fms)
      gb_fm = self.avgpool(gb_fm)
      gb_fm = gb_fm.reshape(gb_fm.size(0),-1)
      concatenated_features = torch.cat([rn_fms, gb_fm], dim=1)
      if not no_attn:
        self.no_attn_status_flag = print_status("Self attention is applied on combined features.", self.no_attn_status_flag)
        concatenated_features = self.combined_attn_layer(concatenated_features)
    elif self.combined=='classification':
      rn_fms = list(rn_fms.values())
      if not self.no_attn:
        self.no_attn_status_flag = print_status("Self attention on Retinanet's feature maps are applied.", self.no_attn_status_flag)
        rn_fms = [self.feature_map_attn(x) for x in rn_fms]
      rn_fms = [self.avgpool(x) for x in rn_fms]
      rn_fms = [x.view(x.size(0), -1) for x in rn_fms]
      concatenated_features = torch.cat(rn_fms, dim = 1)
    return concatenated_features


## Training

In [ ]:
def train_model(gbcret, num_epochs, optimizer, va_curriculum, batch_size, save_model_dir, train_transform, eval_transform,
                freeze_after_20=False, best_acc=0, num_worse_epochs=0, patience=0, checkpoint_save_rate=5,
                last_epoch=0, train_scores={'acc':[],'mAP':[]}, val_scores={'acc':[],'mAP':[]}, lr_scheduler=None):

  # Validation DataLoader
  val_dataset = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, to_blur=False, img_transforms=eval_transform)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn= collate_fn)

  best_epoch = 0
  best_model = None
  show_num_images = True

  os.makedirs(f'{save_model_dir}/checkpoints', exist_ok=True)

  print(f"Last Epoch: {last_epoch}, Start Epoch: {last_epoch+1}, End Epoch: {num_epochs}")
  print(f"VA Curriculum: {va_curriculum}")
  print(f"Train Scores Length: {len(train_scores['acc'])}, Val Scores Length: {len(val_scores['acc'])}")
  print(f"Best accuracy so far: {best_acc}")
  print(f"Patience so far: {num_worse_epochs}")
  for epoch in range(last_epoch+1, num_epochs+1):

    print(f'\nEpoch {epoch}:')
    start_time = time.time()

    if va_curriculum:
      if epoch <= 10:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(65,65), sigma=16, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >10 and epoch <=15:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(33,33), sigma=8, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >15 and epoch <=20:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(17,17), sigma=4, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >20 and epoch <=25:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(9,9), sigma=2, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >25 and epoch <=30:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(5,5), sigma=1, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      else:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=False, img_transforms= train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
    else:
      train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=False, img_transforms=train_transform)
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

    if show_num_images == True:
      print(f"Number of Train images: {len(train_dataset)}")
      print(f"Number of Test images: {len(val_dataset)}")
      show_num_images = False

    # Training Loop
    gbcret.train()
    running_loss = 0.0
    total_step = len(train_loader)
    for i, data in enumerate(train_loader, 0):
      images_ids, inputs, targets = data[0],\
                                    [image.to(device) for image in data[1]],\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      optimizer.zero_grad()

      loss = gbcret(inputs, targets, epoch)

      loss.backward()

      optimizer.step()

      running_loss += loss.item()

    train_loss = running_loss/total_step
    print(f"Train Loss: {train_loss}, Total step: {total_step}")


    # Evaluating Model
    gbcret.eval()
    print("Train Evaluation:")
    train_acc, train_mAP, train_cfm, train_y_true, train_y_pred, train_fnames, _ = evaluate_model(gbcret, train_loader)
    train_scores['acc'].append(train_acc)
    train_scores['mAP'].append(train_mAP)

    print("Val Evaluation:")
    val_acc, val_mAP, val_cfm, val_y_true, val_y_pred, val_fnames, _  = evaluate_model(gbcret, val_loader)
    val_scores['acc'].append(val_acc)
    val_scores['mAP'].append(val_mAP)

    if (lr_scheduler is not None) and (epoch>=50):
      lr_scheduler.step(val_acc)

    # Early Stop
    if val_acc > best_acc:
      best_acc = val_acc
      best_epoch = epoch
      best_model = copy.deepcopy(gbcret)
      num_worse_epochs = 0
    else:
      if patience > 0:
        num_worse_epochs += 1

    if ((epoch) % checkpoint_save_rate == 0) or (num_worse_epochs > patience):
      random_state = random.getstate()
      np_random_state = np.random.get_state()
      torch_random_state = torch.get_rng_state()
      torch.save({'model_state_dict': gbcret.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'best_acc': best_acc,
                  'train_scores': train_scores,
                  'val_scores': val_scores,
                  'epoch': epoch,
                  'num_worse_epochs': num_worse_epochs,
                  'random_state': random_state,
                  'np_random_state': np_random_state,
                  'torch_random_state': torch_random_state},
                  f'{save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth')
      print(f"\nCheckpoint is saved to: {save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth\n")

      if best_model is not None:
        torch.save({'model_state_dict': best_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_acc': best_acc,
                    'epoch': best_epoch},
                    f'{save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f'Best model is saved to: {save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f"Best model was for epoch {best_epoch} with best Acc: {best_acc}")

      else:
        print("\nNo Best model to save!")

      if num_worse_epochs > patience:
        print(f"\nNo better results for last {patience} epochs!\nBest model was for epoch {best_epoch} with best mAP: {best_acc}")
        break


    # Time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Epoch time: {elapsed_time}")
    remaining_epochs = num_epochs - epoch
    remaining_time = round(elapsed_time * remaining_epochs)
    hours, remainder = divmod(remaining_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Estimated time remaining: {hours}:{minutes}:{seconds}")

    print(f"GPU Memory Used: {torch.cuda.memory_allocated()}")

## Experiments

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold0

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 153MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6772822492262895, Total step: 34
Train Evaluation:
mIoU = 0.7763367295265198, median IoU: 0.7987426519393921
TP: 1817, FN: 0, FP: 15
Precision: 0.9918122270742358
Recall: 1.0
Acc: 0.5546 Spec: 0.9547 Sens: 0.2414
Normal vs Not Noraml Acc: 0.6578 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_42.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 59.1MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 42, Start Epoch: 43, End Epoch: 100
VA Curriculum: False
Train Scores Length: 42, Val Scores Length: 42
Best accuracy so far: 0.8480392156862745
Patience so far: 4

Epoch 43:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4700884161626591, Total step: 34
Train Evaluation:
mIoU = 0.8183795809745789, median IoU: 0.8317749500274658
TP: 1827, FN: 0, FP: 5
Precision: 0.9972707423580786
Recall: 1.0
Acc: 0.9148 Spec: 0.9759 Sens: 0.9289
Normal vs Not Noraml Acc: 0.9421 Benign vs Malignant Acc: 0.9413
Val Evaluation:
mIoU = 0.7381638288497925, median IoU: 0.7693983316421509
TP: 201, FN: 0, FP: 3
Precision: 0.9852941176470589
Recall: 1.0
Acc: 0.8627 Spec: 0.9623 Sens: 0.7778
Normal vs Not Noraml Acc: 0.9069 Benign vs Malignant Acc: 0.8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_63.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 81.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 63, Start Epoch: 64, End Epoch: 100
VA Curriculum: False
Train Scores Length: 63, Val Scores Length: 63
Best accuracy so far: 0.8774509803921569
Patience so far: 1

Epoch 64:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.33674149609663906, Total step: 34
Train Evaluation:
mIoU = 0.8271581530570984, median IoU: 0.8401489853858948
TP: 1829, FN: 0, FP: 3
Precision: 0.9983624454148472
Recall: 1.0
Acc: 0.9640 Spec: 0.9912 Sens: 0.9914
Normal vs Not Noraml Acc: 0.9722 Benign vs Malignant Acc: 0.9829
Val Evaluation:
mIoU = 0.7341976165771484, median IoU: 0.7624704837799072
TP: 201, FN: 0, FP: 3
Precision: 0.9852941176470589
Recall: 1.0
Acc: 0.8480 Spec: 1.0000 Sens: 0.6889
Normal vs Not Noraml Acc: 0.8971 Benign vs Malignant Acc: 0.

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_0_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold0: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_84.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 117MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 84, Start Epoch: 85, End Epoch: 100
VA Curriculum: False
Train Scores Length: 84, Val Scores Length: 84
Best accuracy so far: 0.8774509803921569
Patience so far: 22

Epoch 85:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.25507444581564737, Total step: 34
Train Evaluation:
mIoU = 0.838278591632843, median IoU: 0.8494579792022705
TP: 1831, FN: 0, FP: 1
Precision: 0.9994541484716157
Recall: 1.0
Acc: 0.9869 Spec: 0.9985 Sens: 0.9957
Normal vs Not Noraml Acc: 0.9891 Benign vs Malignant Acc: 0.9956
Val Evaluation:
mIoU = 0.7316809296607971, median IoU: 0.7581770420074463
TP: 201, FN: 0, FP: 3
Precision: 0.9852941176470589
Recall: 1.0
Acc: 0.8284 Spec: 0.9937 Sens: 0.6444
Normal vs Not Noraml Acc: 0.8971 Benign vs Malignant Acc: 0.

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold1

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 99.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6817592101938583, Total step: 34
Train Evaluation:
mIoU = 0.7782862186431885, median IoU: 0.797890841960907
TP: 1815, FN: 0, FP: 17
Precision: 0.9907205240174672
Recall: 1.0
Acc: 0.5328 Spec: 0.9539 Sens: 0.2134
Normal vs Not Noraml Acc: 0.6354 Benign vs Mal

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_24.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 65.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 24, Start Epoch: 25, End Epoch: 100
VA Curriculum: False
Train Scores Length: 24, Val Scores Length: 24
Best accuracy so far: 0.8382352941176471
Patience so far: 0

Epoch 25:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5881353301160476, Total step: 34
Train Evaluation:
mIoU = 0.802695631980896, median IoU: 0.8183016180992126
TP: 1821, FN: 0, FP: 11
Precision: 0.993995633187773
Recall: 1.0
Acc: 0.8461 Spec: 0.9408 Sens: 0.9030
Normal vs Not Noraml Acc: 0.8876 Benign vs Malignant Acc: 0.9118
Val Evaluation:
mIoU = 0.731445848941803, median IoU: 0.7741172313690186
TP: 199, FN: 0, FP: 5
Precision: 0.9754901960784313
Recall: 1.0
Acc: 0.8186 Spec: 0.9371 Sens: 0.8444
Normal vs Not Noraml Acc: 0.8824 Benign vs Malignant Acc: 0.852

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:04<00:00, 36.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 100
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.9068627450980392
Patience so far: 2

Epoch 46:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4306777926052318, Total step: 34
Train Evaluation:
mIoU = 0.8177428245544434, median IoU: 0.8312539458274841
TP: 1822, FN: 0, FP: 10
Precision: 0.9945414847161572
Recall: 1.0
Acc: 0.9279 Spec: 0.9846 Sens: 0.9397
Normal vs Not Noraml Acc: 0.9498 Benign vs Malignant Acc: 0.9533
Val Evaluation:
mIoU = 0.7446404695510864, median IoU: 0.7742409706115723
TP: 201, FN: 0, FP: 3
Precision: 0.9852941176470589
Recall: 1.0
Acc: 0.9020 Spec: 0.9811 Sens: 0.8222
Normal vs Not Noraml Acc: 0.9559 Benign vs Malignant Acc: 0.

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_84.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:09<00:00, 16.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 84, Start Epoch: 85, End Epoch: 200
VA Curriculum: False
Train Scores Length: 84, Val Scores Length: 84
Best accuracy so far: 0.9215686274509803
Patience so far: 3

Epoch 85:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.24659648625289693, Total step: 34
Train Evaluation:
mIoU = 0.8370152711868286, median IoU: 0.847678005695343
TP: 1830, FN: 0, FP: 2
Precision: 0.9989082969432315
Recall: 1.0
Acc: 0.9853 Spec: 1.0000 Sens: 0.9957
Normal vs Not Noraml Acc: 0.9864 Benign vs Malignant Acc: 0.9978
Val Evaluation:
mIoU = 0.7386377453804016, median IoU: 0.7682034373283386
TP: 201, FN: 0, FP: 3
Precision: 0.9852941176470589
Recall: 1.0
Acc: 0.9020 Spec: 1.0000 Sens: 0.7778
Normal vs Not Noraml Acc: 0.9510 Benign vs Malignant Acc: 0.8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_129.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 85.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 129, Start Epoch: 130, End Epoch: 200
VA Curriculum: False
Train Scores Length: 129, Val Scores Length: 129
Best accuracy so far: 0.9313725490196079
Patience so far: 23

Epoch 130:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.18141016013482036, Total step: 34
Train Evaluation:
mIoU = 0.8568459749221802, median IoU: 0.8672323822975159
TP: 1832, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9973 Spec: 0.9993 Sens: 1.0000
Normal vs Not Noraml Acc: 0.9978 Benign vs Malignant Acc: 0.9989
Val Evaluation:
mIoU = 0.745627760887146, median IoU: 0.780652642250061
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.9020 Spec: 1.0000 Sens: 0.8444
Normal vs Not Noraml Acc: 0.9363 Benign vs Malignant Acc: 0.9231
Epoch 

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_1_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold1: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_150.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 132MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 150, Start Epoch: 151, End Epoch: 200
VA Curriculum: False
Train Scores Length: 150, Val Scores Length: 150
Best accuracy so far: 0.9362745098039216
Patience so far: 14

Epoch 151:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.1746972992139704, Total step: 34
Train Evaluation:
mIoU = 0.8635737299919128, median IoU: 0.8767728805541992
TP: 1832, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9984 Spec: 1.0000 Sens: 0.9978
Normal vs Not Noraml Acc: 0.9989 Benign vs Malignant Acc: 0.9989
Val Evaluation:
mIoU = 0.7453664541244507, median IoU: 0.7759464979171753
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.9118 Spec: 0.9937 Sens: 0.8889
Normal vs Not Noraml Acc: 0.9412 Benign vs Malignant Acc: 0.9333
Epoch

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold2

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 96.2MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6779331775272595, Total step: 34
Train Evaluation:
mIoU = 0.7789933681488037, median IoU: 0.7994398474693298
TP: 1816, FN: 0, FP: 16
Precision: 0.9912663755458515
Recall: 1.0
Acc: 0.5295 Spec: 0.9512 Sens: 0.1808
Normal vs Not Noraml Acc: 0.6332 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_48.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 132MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 48, Start Epoch: 49, End Epoch: 100
VA Curriculum: False
Train Scores Length: 48, Val Scores Length: 48
Best accuracy so far: 0.8823529411764706
Patience so far: 1

Epoch 49:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4238344746477464, Total step: 34
Train Evaluation:
mIoU = 0.8233145475387573, median IoU: 0.8355641961097717
TP: 1826, FN: 0, FP: 6
Precision: 0.9967248908296943
Recall: 1.0
Acc: 0.9209 Spec: 0.9832 Sens: 0.9368
Normal vs Not Noraml Acc: 0.9460 Benign vs Malignant Acc: 0.9459
Val Evaluation:
mIoU = 0.7445062398910522, median IoU: 0.7804350852966309
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.8775 Spec: 0.9870 Sens: 0.7800
Normal vs Not Noraml Acc: 0.9216 Benign vs Malignant Acc: 0.9

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_2_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold2: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_99.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 69.2MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 99, Start Epoch: 100, End Epoch: 200
VA Curriculum: False
Train Scores Length: 99, Val Scores Length: 99
Best accuracy so far: 0.9215686274509803
Patience so far: 14

Epoch 100:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.22535882232820287, Total step: 34
Train Evaluation:
mIoU = 0.8436225056648254, median IoU: 0.8543511629104614
TP: 1829, FN: 0, FP: 3
Precision: 0.9983624454148472
Recall: 1.0
Acc: 0.9896 Spec: 0.9978 Sens: 0.9956
Normal vs Not Noraml Acc: 0.9924 Benign vs Malignant Acc: 0.9945
Val Evaluation:
mIoU = 0.7440788745880127, median IoU: 0.7770698666572571
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.9314 Spec: 1.0000 Sens: 0.9200
Normal vs Not Noraml Acc: 0.9510 Benign vs Malignant Acc:

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold3

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 123MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6786081825985628, Total step: 34
Train Evaluation:
mIoU = 0.7793318629264832, median IoU: 0.8016136884689331
TP: 1817, FN: 0, FP: 15
Precision: 0.9918122270742358
Recall: 1.0
Acc: 0.5590 Spec: 0.9607 Sens: 0.1895
Normal vs Not Noraml Acc: 0.6561 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_21.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 142MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 21, Start Epoch: 22, End Epoch: 100
VA Curriculum: False
Train Scores Length: 21, Val Scores Length: 21
Best accuracy so far: 0.8382352941176471
Patience so far: 0

Epoch 22:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.6224457481328178, Total step: 34
Train Evaluation:
mIoU = 0.8037609457969666, median IoU: 0.8196229338645935
TP: 1820, FN: 0, FP: 12
Precision: 0.9934497816593887
Recall: 1.0
Acc: 0.8319 Spec: 0.9330 Sens: 0.8889
Normal vs Not Noraml Acc: 0.8772 Benign vs Malignant Acc: 0.9027
Val Evaluation:
mIoU = 0.7464236617088318, median IoU: 0.762800395488739
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.8382 Spec: 0.9091 Sens: 0.8800
Normal vs Not Noraml Acc: 0.9020 Benign vs Malignant Acc: 0.8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_3_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold3: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_72.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 50.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 72, Start Epoch: 73, End Epoch: 100
VA Curriculum: False
Train Scores Length: 72, Val Scores Length: 72
Best accuracy so far: 0.9215686274509803
Patience so far: 25

Epoch 73:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.28272459743654027, Total step: 34
Train Evaluation:
mIoU = 0.8298864960670471, median IoU: 0.8421043753623962
TP: 1826, FN: 0, FP: 6
Precision: 0.9967248908296943
Recall: 1.0
Acc: 0.9749 Spec: 0.9971 Sens: 0.9891
Normal vs Not Noraml Acc: 0.9793 Benign vs Malignant Acc: 0.9909
Val Evaluation:
mIoU = 0.7487503290176392, median IoU: 0.7610197067260742
TP: 202, FN: 0, FP: 2
Precision: 0.9901960784313726
Recall: 1.0
Acc: 0.8725 Spec: 0.9870 Sens: 0.8600
Normal vs Not Noraml Acc: 0.9118 Benign vs Malignant Acc: 0

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold4

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 90.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6800366675152498, Total step: 34
Train Evaluation:
mIoU = 0.7807288765907288, median IoU: 0.802514374256134
TP: 1814, FN: 0, FP: 18
Precision: 0.990174672489083
Recall: 1.0
Acc: 0.5404 Spec: 0.9716 Sens: 0.1432
Normal vs Not Noraml Acc: 0.6365 Benign vs Mali

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_21.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 142MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 21, Start Epoch: 22, End Epoch: 100
VA Curriculum: False
Train Scores Length: 21, Val Scores Length: 21
Best accuracy so far: 0.7990196078431373
Patience so far: 0

Epoch 22:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.6393270965884713, Total step: 34
Train Evaluation:
mIoU = 0.8013433814048767, median IoU: 0.820764422416687
TP: 1818, FN: 0, FP: 14
Precision: 0.99235807860262
Recall: 1.0
Acc: 0.8368 Spec: 0.9373 Sens: 0.8915
Normal vs Not Noraml Acc: 0.8837 Benign vs Malignant Acc: 0.8992
Val Evaluation:
mIoU = 0.7562023997306824, median IoU: 0.7774321436882019
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.8137 Spec: 0.9744 Sens: 0.8750
Normal vs Not Noraml Acc: 0.8431 Benign vs Malignant Acc: 0.9333
Epoch time: 2

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_42.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 115MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 42, Start Epoch: 43, End Epoch: 100
VA Curriculum: False
Train Scores Length: 42, Val Scores Length: 42
Best accuracy so far: 0.8676470588235294
Patience so far: 3

Epoch 43:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.48596494075130015, Total step: 34
Train Evaluation:
mIoU = 0.8180403709411621, median IoU: 0.8303149938583374
TP: 1825, FN: 0, FP: 7
Precision: 0.99617903930131
Recall: 1.0
Acc: 0.9192 Spec: 0.9788 Sens: 0.9306
Normal vs Not Noraml Acc: 0.9465 Benign vs Malignant Acc: 0.9412
Val Evaluation:
mIoU = 0.7609037756919861, median IoU: 0.7729445099830627
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.8627 Spec: 0.9808 Sens: 0.8542
Normal vs Not Noraml Acc: 0.9069 Benign vs Malignant Acc: 0.8977
Epoch time: 

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_93.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:11<00:00, 13.1MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 93, Start Epoch: 94, End Epoch: 100
VA Curriculum: False
Train Scores Length: 93, Val Scores Length: 93
Best accuracy so far: 0.9411764705882353
Patience so far: 0

Epoch 94:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.2331320485647987, Total step: 34
Train Evaluation:
mIoU = 0.8394447565078735, median IoU: 0.8502880930900574
TP: 1831, FN: 0, FP: 1
Precision: 0.9994541484716157
Recall: 1.0
Acc: 0.9907 Spec: 0.9993 Sens: 0.9957
Normal vs Not Noraml Acc: 0.9924 Benign vs Malignant Acc: 0.9967
Val Evaluation:
mIoU = 0.7649793028831482, median IoU: 0.7836056351661682
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9216 Spec: 0.9872 Sens: 0.9167
Normal vs Not Noraml Acc: 0.9510 Benign vs Malignant Acc: 0.9375
Epoch time:

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_99.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 66.0MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 99, Start Epoch: 100, End Epoch: 200
VA Curriculum: False
Train Scores Length: 99, Val Scores Length: 99
Best accuracy so far: 0.9411764705882353
Patience so far: 6

Epoch 100:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.22082039670032613, Total step: 34
Train Evaluation:
mIoU = 0.8447133302688599, median IoU: 0.8545566201210022
TP: 1831, FN: 0, FP: 1
Precision: 0.9994541484716157
Recall: 1.0
Acc: 0.9896 Spec: 1.0000 Sens: 0.9935
Normal vs Not Noraml Acc: 0.9913 Benign vs Malignant Acc: 0.9967
Val Evaluation:
mIoU = 0.7627221345901489, median IoU: 0.7820883393287659
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9118 Spec: 0.9872 Sens: 0.8958
Normal vs Not Noraml Acc: 0.9461 Benign vs Malignant Acc: 0.9278
Epoch ti

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_114.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 137MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 114, Start Epoch: 115, End Epoch: 200
VA Curriculum: False
Train Scores Length: 114, Val Scores Length: 114
Best accuracy so far: 0.946078431372549
Patience so far: 3

Epoch 115:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.2085770951474414, Total step: 34
Train Evaluation:
mIoU = 0.8518003821372986, median IoU: 0.862991213798523
TP: 1831, FN: 0, FP: 1
Precision: 0.9994541484716157
Recall: 1.0
Acc: 0.9940 Spec: 0.9993 Sens: 0.9957
Normal vs Not Noraml Acc: 0.9956 Benign vs Malignant Acc: 0.9967
Val Evaluation:
mIoU = 0.7608166933059692, median IoU: 0.7809056043624878
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9363 Spec: 0.9872 Sens: 0.9583
Normal vs Not Noraml Acc: 0.9559 Benign vs Malignant Acc: 0.9596
Epoch ti

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_135.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 100MB/s] 


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 135, Start Epoch: 136, End Epoch: 200
VA Curriculum: False
Train Scores Length: 135, Val Scores Length: 135
Best accuracy so far: 0.946078431372549
Patience so far: 24

Epoch 136:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.1775369232191759, Total step: 34
Train Evaluation:
mIoU = 0.8593607544898987, median IoU: 0.8681920766830444
TP: 1832, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9967 Spec: 0.9993 Sens: 1.0000
Normal vs Not Noraml Acc: 0.9973 Benign vs Malignant Acc: 0.9989
Val Evaluation:
mIoU = 0.7670249342918396, median IoU: 0.7890702486038208
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9265 Spec: 0.9808 Sens: 0.9583
Normal vs Not Noraml Acc: 0.9510 Benign vs Malignant Acc: 0.9490
Epoch time: 264.06211

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_4_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold4: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_159.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 106MB/s] 


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 159, Start Epoch: 160, End Epoch: 200
VA Curriculum: False
Train Scores Length: 159, Val Scores Length: 159
Best accuracy so far: 0.9558823529411765
Patience so far: 22

Epoch 160:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.1685650423169136, Total step: 34
Train Evaluation:
mIoU = 0.8674317598342896, median IoU: 0.8760921955108643
TP: 1832, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9984 Spec: 1.0000 Sens: 1.0000
Normal vs Not Noraml Acc: 0.9984 Benign vs Malignant Acc: 1.0000
Val Evaluation:
mIoU = 0.7659459114074707, median IoU: 0.7823228240013123
TP: 204, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.9363 Spec: 0.9808 Sens: 0.9583
Normal vs Not Noraml Acc: 0.9608 Benign vs Malignant Acc: 0.9479
Epoch time: 252.5209

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold5

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 79.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.678197881754707, Total step: 34
Train Evaluation:
mIoU = 0.7805460691452026, median IoU: 0.8023818135261536
TP: 1817, FN: 0, FP: 15
Precision: 0.9918122270742358
Recall: 1.0
Acc: 0.5606 Spec: 0.9465 Sens: 0.2655
Normal vs Not Noraml Acc: 0.6687 Benign vs Mal

KeyboardInterrupt: 

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_36.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 145MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 36, Start Epoch: 37, End Epoch: 100
VA Curriculum: False
Train Scores Length: 36, Val Scores Length: 36
Best accuracy so far: 0.8627450980392157
Patience so far: 3

Epoch 37:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5259572469136294, Total step: 34
Train Evaluation:
mIoU = 0.8118324875831604, median IoU: 0.8266159892082214
TP: 1821, FN: 0, FP: 11
Precision: 0.993995633187773
Recall: 1.0
Acc: 0.9121 Spec: 0.9758 Sens: 0.9272
Normal vs Not Noraml Acc: 0.9367 Benign vs Malignant Acc: 0.9472
Val Evaluation:
mIoU = 0.7260806560516357, median IoU: 0.7711009383201599
TP: 198, FN: 0, FP: 6
Precision: 0.9705882352941176
Recall: 1.0
Acc: 0.8333 Spec: 0.9877 Sens: 0.7619
Normal vs Not Noraml Acc: 0.8922 Benign vs Malignant Acc: 0.8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_5_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold5: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 88.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 200
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.8627450980392157
Patience so far: 12

Epoch 46:
Number of Train images: 1832
Number of Test images: 204
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.42890529159237356, Total step: 34
Train Evaluation:
mIoU = 0.8198384642601013, median IoU: 0.8306723833084106
TP: 1822, FN: 0, FP: 10
Precision: 0.9945414847161572
Recall: 1.0
Acc: 0.9334 Spec: 0.9846 Sens: 0.9507
Normal vs Not Noraml Acc: 0.9525 Benign vs Malignant Acc: 0.9591
Val Evaluation:
mIoU = 0.7285078763961792, median IoU: 0.7725805044174194
TP: 200, FN: 0, FP: 4
Precision: 0.9803921568627451
Recall: 1.0
Acc: 0.8431 Spec: 0.9815 Sens: 0.7619
Normal vs Not Noraml Acc: 0.9020 Benign vs Malignant Acc: 

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold6

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 82.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6758378148078918, Total step: 34
Train Evaluation:
mIoU = 0.7826818227767944, median IoU: 0.8035234808921814
TP: 1817, FN: 0, FP: 16
Precision: 0.9912711402073104
Recall: 1.0
Acc: 0.5576 Spec: 0.9725 Sens: 0.1578
Normal vs Not Noraml Acc: 0.6410 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 143MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 15, Start Epoch: 16, End Epoch: 100
VA Curriculum: False
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.5665024630541872
Patience so far: 4

Epoch 16:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.7501480807276333, Total step: 34
Train Evaluation:
mIoU = 0.7958386540412903, median IoU: 0.814125657081604
TP: 1819, FN: 0, FP: 14
Precision: 0.9923622476813966
Recall: 1.0
Acc: 0.6999 Spec: 0.8142 Sens: 0.9289
Normal vs Not Noraml Acc: 0.7763 Benign vs Malignant Acc: 0.8398
Val Evaluation:
mIoU = 0.722758412361145, median IoU: 0.7564214468002319
TP: 200, FN: 0, FP: 3
Precision: 0.9852216748768473
Recall: 1.0
Acc: 0.7488 Spec: 0.8125 Sens: 0.9492
Normal vs Not Noraml Acc: 0.7931 Benign vs Malignant Acc: 0.91

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_36.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 83.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 36, Start Epoch: 37, End Epoch: 100
VA Curriculum: False
Train Scores Length: 36, Val Scores Length: 36
Best accuracy so far: 0.9113300492610837
Patience so far: 0

Epoch 37:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5214362714220496, Total step: 34
Train Evaluation:
mIoU = 0.816926121711731, median IoU: 0.8296791911125183
TP: 1822, FN: 0, FP: 11
Precision: 0.993998908892526
Recall: 1.0
Acc: 0.9062 Spec: 0.9732 Sens: 0.9222
Normal vs Not Noraml Acc: 0.9329 Benign vs Malignant Acc: 0.9419
Val Evaluation:
mIoU = 0.7290689945220947, median IoU: 0.7604567408561707
TP: 200, FN: 0, FP: 3
Precision: 0.9852216748768473
Recall: 1.0
Acc: 0.9163 Spec: 0.9861 Sens: 0.8814
Normal vs Not Noraml Acc: 0.9261 Benign vs Malignant Acc: 0.97

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_6_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold6: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_57.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:21<00:00, 7.10MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 57, Start Epoch: 58, End Epoch: 100
VA Curriculum: False
Train Scores Length: 57, Val Scores Length: 57
Best accuracy so far: 0.9211822660098522
Patience so far: 16

Epoch 58:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.3231411361519028, Total step: 34
Train Evaluation:
mIoU = 0.8250272870063782, median IoU: 0.8383718729019165
TP: 1826, FN: 0, FP: 7
Precision: 0.9961811238406983
Recall: 1.0
Acc: 0.9613 Spec: 0.9942 Sens: 0.9756
Normal vs Not Noraml Acc: 0.9705 Benign vs Malignant Acc: 0.9805
Val Evaluation:
mIoU = 0.7369600534439087, median IoU: 0.757281482219696
TP: 201, FN: 0, FP: 2
Precision: 0.9901477832512315
Recall: 1.0
Acc: 0.9015 Spec: 1.0000 Sens: 0.8983
Normal vs Not Noraml Acc: 0.9261 Benign vs Malignant Acc: 0.9

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold7

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 139MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6774470175013823, Total step: 34
Train Evaluation:
mIoU = 0.7787815928459167, median IoU: 0.8008307814598083
TP: 1815, FN: 0, FP: 18
Precision: 0.9901800327332242
Recall: 1.0
Acc: 0.5565 Spec: 0.9682 Sens: 0.1205
Normal vs Not Noraml Acc: 0.6394 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_21.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 40.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 21, Start Epoch: 22, End Epoch: 100
VA Curriculum: False
Train Scores Length: 21, Val Scores Length: 21
Best accuracy so far: 0.7980295566502463
Patience so far: 0

Epoch 22:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.6510895490646362, Total step: 34
Train Evaluation:
mIoU = 0.8008772134780884, median IoU: 0.8171316385269165
TP: 1819, FN: 0, FP: 14
Precision: 0.9923622476813966
Recall: 1.0
Acc: 0.8538 Spec: 0.9386 Sens: 0.8996
Normal vs Not Noraml Acc: 0.8974 Benign vs Malignant Acc: 0.9054
Val Evaluation:
mIoU = 0.7427512407302856, median IoU: 0.7725502252578735
TP: 200, FN: 0, FP: 3
Precision: 0.9852216748768473
Recall: 1.0
Acc: 0.8079 Spec: 0.9155 Sens: 0.8689
Normal vs Not Noraml Acc: 0.8719 Benign vs Malignant Acc: 0.

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_42.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 98.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 42, Start Epoch: 43, End Epoch: 100
VA Curriculum: False
Train Scores Length: 42, Val Scores Length: 42
Best accuracy so far: 0.8669950738916257
Patience so far: 0

Epoch 43:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4753388306673835, Total step: 34
Train Evaluation:
mIoU = 0.820427656173706, median IoU: 0.8322604298591614
TP: 1823, FN: 0, FP: 10
Precision: 0.994544462629569
Recall: 1.0
Acc: 0.9236 Spec: 0.9812 Sens: 0.9330
Normal vs Not Noraml Acc: 0.9454 Benign vs Malignant Acc: 0.9530
Val Evaluation:
mIoU = 0.7457218170166016, median IoU: 0.7735630869865417
TP: 200, FN: 0, FP: 3
Precision: 0.9852216748768473
Recall: 1.0
Acc: 0.8621 Spec: 0.9648 Sens: 0.9016
Normal vs Not Noraml Acc: 0.8966 Benign vs Malignant Acc: 0.93

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_63.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 78.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 63, Start Epoch: 64, End Epoch: 100
VA Curriculum: False
Train Scores Length: 63, Val Scores Length: 63
Best accuracy so far: 0.8866995073891626
Patience so far: 3

Epoch 64:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.31712699067943234, Total step: 34
Train Evaluation:
mIoU = 0.8278135061264038, median IoU: 0.840798020362854
TP: 1827, FN: 0, FP: 6
Precision: 0.9967266775777414
Recall: 1.0
Acc: 0.9640 Spec: 0.9942 Sens: 0.9844
Normal vs Not Noraml Acc: 0.9716 Benign vs Malignant Acc: 0.9839
Val Evaluation:
mIoU = 0.7489004731178284, median IoU: 0.7704533934593201
TP: 202, FN: 0, FP: 1
Precision: 0.9950738916256158
Recall: 1.0
Acc: 0.8867 Spec: 0.9930 Sens: 0.8852
Normal vs Not Noraml Acc: 0.9163 Benign vs Malignant Acc: 0.9

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_7_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold7: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_84.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 146MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 84, Start Epoch: 85, End Epoch: 100
VA Curriculum: False
Train Scores Length: 84, Val Scores Length: 84
Best accuracy so far: 0.9014778325123153
Patience so far: 18

Epoch 85:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.2563545624999439, Total step: 34
Train Evaluation:
mIoU = 0.8404951691627502, median IoU: 0.8515522480010986
TP: 1829, FN: 0, FP: 4
Precision: 0.9978177850518276
Recall: 1.0
Acc: 0.9842 Spec: 1.0000 Sens: 0.9888
Normal vs Not Noraml Acc: 0.9869 Benign vs Malignant Acc: 0.9944
Val Evaluation:
mIoU = 0.7534137964248657, median IoU: 0.7730923295021057
TP: 202, FN: 0, FP: 1
Precision: 0.9950738916256158
Recall: 1.0
Acc: 0.8621 Spec: 0.9859 Sens: 0.8197
Normal vs Not Noraml Acc: 0.9163 Benign vs Malignant Acc: 0.

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold8

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 142MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.675698459148407, Total step: 34
Train Evaluation:
mIoU = 0.7826324105262756, median IoU: 0.8020827174186707
TP: 1816, FN: 0, FP: 17
Precision: 0.9907255864702673
Recall: 1.0
Acc: 0.5412 Spec: 0.9615 Sens: 0.1780
Normal vs Not Noraml Acc: 0.6307 Benign vs Mal

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_8_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold8: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_60.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:00<00:00, 156MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 60, Start Epoch: 61, End Epoch: 100
VA Curriculum: False
Train Scores Length: 60, Val Scores Length: 60
Best accuracy so far: 0.9261083743842364
Patience so far: 12

Epoch 61:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.32985753346891966, Total step: 34
Train Evaluation:
mIoU = 0.8246790766716003, median IoU: 0.8376423716545105
TP: 1826, FN: 0, FP: 7
Precision: 0.9961811238406983
Recall: 1.0
Acc: 0.9656 Spec: 0.9956 Sens: 0.9934
Normal vs Not Noraml Acc: 0.9705 Benign vs Malignant Acc: 0.9897
Val Evaluation:
mIoU = 0.7404556274414062, median IoU: 0.7617177963256836
TP: 203, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Acc: 0.8867 Spec: 0.9732 Sens: 0.8704
Normal vs Not Noraml Acc: 0.9409 Benign vs Malignant Acc: 0.8878
Epoch tim

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3", Fold9

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 137MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.6757174870547127, Total step: 34
Train Evaluation:
mIoU = 0.7800768613815308, median IoU: 0.7998923659324646
TP: 1819, FN: 0, FP: 14
Precision: 0.9923622476813966
Recall: 1.0
Acc: 0.5576 Spec: 0.9623 Sens: 0.1674
Normal vs Not Noraml Acc: 0.6432 Benign vs Ma

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_21.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 147MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 21, Start Epoch: 22, End Epoch: 100
VA Curriculum: False
Train Scores Length: 21, Val Scores Length: 21
Best accuracy so far: 0.8620689655172413
Patience so far: 0

Epoch 22:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.6393320122185875, Total step: 34
Train Evaluation:
mIoU = 0.8025044202804565, median IoU: 0.8155164122581482
TP: 1825, FN: 0, FP: 8
Precision: 0.9956355701036552
Recall: 1.0
Acc: 0.8456 Spec: 0.9297 Sens: 0.8943
Normal vs Not Noraml Acc: 0.8920 Benign vs Malignant Acc: 0.8988
Val Evaluation:
mIoU = 0.7373965382575989, median IoU: 0.7713714838027954
TP: 198, FN: 0, FP: 5
Precision: 0.9753694581280788
Recall: 1.0
Acc: 0.8571 Spec: 0.9527 Sens: 0.9091
Normal vs Not Noraml Acc: 0.9015 Benign vs Malignant Acc: 0.9

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_42.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 147MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 42, Start Epoch: 43, End Epoch: 100
VA Curriculum: False
Train Scores Length: 42, Val Scores Length: 42
Best accuracy so far: 0.9556650246305419
Patience so far: 0

Epoch 43:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.45359116266755495, Total step: 34
Train Evaluation:
mIoU = 0.8182782530784607, median IoU: 0.8331252336502075
TP: 1824, FN: 0, FP: 9
Precision: 0.9950900163666121
Recall: 1.0
Acc: 0.9154 Spec: 0.9848 Sens: 0.9229
Normal vs Not Noraml Acc: 0.9367 Benign vs Malignant Acc: 0.9533
Val Evaluation:
mIoU = 0.7395347356796265, median IoU: 0.7816975116729736
TP: 197, FN: 0, FP: 6
Precision: 0.9704433497536946
Recall: 1.0
Acc: 0.9409 Spec: 0.9865 Sens: 0.9091
Normal vs Not Noraml Acc: 0.9655 Benign vs Malignant Acc: 0.

In [ ]:
train_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_train.txt"
val_txt_path = "/content/drive/MyDrive/CV/Project/data/CV splits/fold_9_test.txt"

lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/CV/Fold9: GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_63.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 49.2MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 63, Start Epoch: 64, End Epoch: 100
VA Curriculum: False
Train Scores Length: 63, Val Scores Length: 63
Best accuracy so far: 0.9556650246305419
Patience so far: 21

Epoch 64:
Number of Train images: 1833
Number of Test images: 203
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.3418525834293926, Total step: 34
Train Evaluation:
mIoU = 0.8251798748970032, median IoU: 0.8380964398384094
TP: 1827, FN: 0, FP: 6
Precision: 0.9967266775777414
Recall: 1.0
Acc: 0.9607 Spec: 0.9942 Sens: 0.9758
Normal vs Not Noraml Acc: 0.9705 Benign vs Malignant Acc: 0.9793
Val Evaluation:
mIoU = 0.7388989329338074, median IoU: 0.7791146636009216
TP: 198, FN: 0, FP: 5
Precision: 0.9753694581280788
Recall: 1.0
Acc: 0.9409 Spec: 0.9932 Sens: 0.8545
Normal vs Not Noraml Acc: 0.9754 Benign vs Malignant Acc: 0.

# End